In [ ]:
import os
import pandas as pd

# Set up working directory
BASE_PATH = "Z:/File folders/Teaching/Reproducible Research/2023/Repository/RRcourse2023/6. Coding and documentation"
os.chdir(BASE_PATH)

# Load O*NET task data
task_data = pd.read_csv("Data/onet_tasks.csv")

# Load Eurostat employment data sheets
def load_eurostat_data():
    """Load Eurostat employment data from Excel sheets"""
    sheets = ["ISCO1", "ISCO2", "ISCO3", "ISCO4", 
             "ISCO5", "ISCO6", "ISCO7", "ISCO8", "ISCO9"]
    
    # Dictionary to store loaded DataFrames
    employment_data = {}
    for sheet_name in sheets:
        df = pd.read_excel(
            "Data/Eurostat_employment_isco.xlsx",
            sheet_name=sheet_name
        )
        employment_data[sheet_name] = df
        
    return employment_data

# Load all sheets into memory
employment_data = load_eurostat_data()

def calculate_country_totals(country_name):
    """Calculate total workers for a country across all ISCO levels"""
    total_workers = sum(df[country_name] 
                       for df in employment_data.values() 
                       if country_name in df.columns)
    return total_workers

# Calculate totals for selected countries
countries_of_interest = ["Belgium", "Spain", "Poland"]
country_totals = {
    country: calculate_country_totals(country)
    for country in countries_of_interest
}

In [ ]:
import pandas as pd

def assign_isco_categories(dataframes):
    """
    Assign ISCO category numbers to each dataframe.
    
    Args:
        dataframes (list): List of pandas DataFrames containing employment data
        
    Returns:
        dict: Dictionary mapping ISCO levels to their corresponding DataFrames
    """
    return {f"isco{i}": df.assign(ISCO=i) 
            for i, df in enumerate(dataframes, 1)}

def merge_datasets(dataframes_dict):
    """
    Merge all datasets vertically into a single DataFrame.
    
    Args:
        dataframes_dict (dict): Dictionary of DataFrames with ISCO assignments
        
    Returns:
        DataFrame: Concatenated dataset with ISCO category information
    """
    return pd.concat(dataframes_dict.values(), ignore_index=True)

def replicate_totals(all_data, totals_dict):
    """
    Replicate country totals across ISCO categories.
    
    Args:
        all_data (DataFrame): Concatenated dataset
        totals_dict (dict): Dictionary of country totals
        
    Returns:
        DataFrame: Dataset with replicated totals
    """
    for country in totals_dict.keys():
        all_data[f"total_{country}"] = pd.concat([totals_dict[country]] * 9, 
                                               ignore_index=True)
    return all_data

def calculate_shares(dataset):
    """
    Calculate occupation shares for each country.
    
    Args:
        dataset (DataFrame): Dataset with employment and total data
        
    Returns:
        DataFrame: Dataset with share calculations added
    """
    countries = ['Belgium', 'Spain', 'Poland']
    for country in countries:
        dataset[f'share_{country.lower()}'] = (
            dataset[country] / dataset[f'total_{country}']
        )
    return dataset

# Process the datasets
isco_dataframes = [isco1, isco2, isco3, isco4, isco5, isco6, isco7, isco8, isco9]

# Step 1: Assign ISCO categories
isco_with_categories = assign_isco_categories(isco_dataframes)

# Step 2: Merge all datasets
merged_data = merge_datasets(isco_with_categories)

# Step 3: Add country totals
merged_data = replicate_totals(merged_data, {
    'Belgium': total_Belgium,
    'Spain': total_Spain,
    'Poland': total_Poland
})

# Step 4: Calculate shares
final_data = calculate_shares(merged_data)

In [ ]:
# Now let's look at the task data. We want the first digit of the ISCO variable only
import pandas as pd
import numpy as np
import re

task_data["isco08_1dig"] = task_data["isco08"].astype(str).str[:1].astype(int)

# And we'll calculate the mean task values at a 1-digit level 
# (more on what these tasks are below)
aggdata = task_data.groupby(["isco08_1dig"]).mean()
aggdata = aggdata.drop(columns=["isco08"])

# We'll be interested in tracking the intensity of Non-routine cognitive analytical tasks
# Using a framework reminiscent of the work by David Autor.

#These are the ones we're interested in:
# Non-routine cognitive analytical
# 4.A.2.a.4 Analyzing Data or Information
# 4.A.2.b.2 Thinking Creatively
# 4.A.4.a.1 Interpreting the Meaning of Information for Others

#Let's combine the data.
combined = pd.merge(all_data, aggdata, left_on='ISCO', right_on='isco08_1dig', how='left')
# Traditionally, the first step is to standardise the task values using weights 
# defined by share of occupations in the labour force. This should be done separately
# for each country. Standardisation -> getting the mean to 0 and std. dev. to 1.
# Let's do this for each of the variables that interests us:

In [ ]:
from scipy.stats import gmean
import numpy as np

def calculate_standardized_values(data, column, weight_column):
    """
    Calculate standardized values for a given column using weighted statistics.
    
    Args:
        data (DataFrame): Input dataset
        column (str): Column containing values to standardize
        weight_column (str): Column containing weights
        
    Returns:
        Series: Standardized values
    """
    mean = np.average(data[column], weights=data[weight_column])
    std_dev = np.sqrt(np.average((data[column] - mean)**2, 
                                weights=data[weight_column]))
    return (data[column] - mean) / std_dev

def process_task_items(data):
    """
    Process standardized calculations for all task items and countries.
    
    Args:
        data (DataFrame): Input dataset containing task items and weights
        
    Returns:
        DataFrame: Dataset with standardized columns added
    """
    # Define task items and countries
    task_items = ["t_4A2a4", "t_4A2b2", "t_4A4a1"]
    countries = ["Belgium", "Poland", "Spain"]
    
    # Process each task item
    for task_item in task_items:
        # Process each country
        for country in countries:
            weight_column = f"share_{country}"
            result_column = f"std_{country}_t_{task_item}"
            
            # Calculate standardized values
            data[result_column] = calculate_standardized_values(
                data, task_item, weight_column
            )
    
    return data

# Process all task items
combined = process_task_items(combined)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_nrca_scores(data, country):
    """
    Calculate Non-Routine Cognitive Analytical (NRCA) task scores for a country.
    
    Args:
        data (DataFrame): Input dataset
        country (str): Country name (e.g., "Belgium", "Poland", "Spain")
        
    Returns:
        Series: Combined NRCA scores
    """
    task_columns = [
        f"std_{country}_t_4A2a4",
        f"std_{country}_t_4A2b2",
        f"std_{country}_t_4A4a1"
    ]
    return data[task_columns].sum(axis=1)

def standardize_values(data, column, weight_column):
    """
    Standardize values using weighted statistics.
    
    Args:
        data (DataFrame): Input dataset
        column (str): Column to standardize
        weight_column (str): Column containing weights
        
    Returns:
        Series: Standardized values
    """
    mean = np.average(data[column], weights=data[weight_column])
    std_dev = np.sqrt(np.average((data[column] - mean)**2, 
                                weights=data[weight_column]))
    return (data[column] - mean) / std_dev

def calculate_country_aggregates(data, country):
    """
    Calculate time-series aggregates for a country's NRCA scores.
    
    Args:
        data (DataFrame): Input dataset
        country (str): Country name
        
    Returns:
        DataFrame: Aggregated time-series data
    """
    # Calculate NRCA scores
    data[f"{country}_NRCA"] = calculate_nrca_scores(data, country)
    
    # Standardize scores
    data[f"std_{country}_NRCA"] = standardize_values(
        data, f"{country}_NRCA", f"share_{country}"
    )
    
    # Calculate weighted aggregates
    data[f"multip_{country}_NRCA"] = (
        data[f"std_{country}_NRCA"] * data[f"share_{country}"]
    )
    
    # Aggregate by time
    return data.groupby("TIME")[f"multip_{country}_NRCA"].sum().reset_index()

def plot_nrca_trends(data, country):
    """
    Plot NRCA trends over time for a country.
    
    Args:
        data (DataFrame): Aggregated time-series data
        country (str): Country name
    """
    plt.figure(figsize=(12, 6))
    plt.plot(data["TIME"], data[f"multip_{country}_NRCA"])
    plt.xticks(range(0, len(data), 3), data["TIME"][::3])
    plt.title(f"NRCA Trends for {country}")
    plt.xlabel("Time")
    plt.ylabel("Weighted NRCA Score")
    plt.grid(True)
    plt.show()

# Process data for each country
countries = ["Belgium", "Poland", "Spain"]
for country in countries:
    aggregates = calculate_country_aggregates(combined, country)
    plot_nrca_trends(aggregates, country)